# Using the Apply Method on a DataFrame

This example is based on a stackoverflow post here:
https://stackoverflow.com/questions/61602609/pandas-dict-of-columns-and-select-the-right-value

In [1]:
import pandas as pd
import numpy as np
from itertools import chain

# Suppose you have a pandas dataframe like this:
columns = ["Book_Name", "Book_Ids", "Countries_Published"]
data = np.array([
['Book_A',             [123],               ['USA']],
['Book_B',             [4670,1122],         ['Canada', 'USA']],
['Book_C',             [11,200,7688],       ['Japan', 'USA', 'Canada']],
['Book_D',             [500,400,600],       ['NaN', 'Japan', 'Canada']],
['Book_E',             [987],               ['Japan']],
])
df = pd.DataFrame(data, columns=columns)
df

,Book_Name,Book_Ids,Countries_Published
0,Book_A,[123],[USA]
1,Book_B,"[4670, 1122]","[Canada, USA]"
2,Book_C,"[11, 200, 7688]","[Japan, USA, Canada]"
3,Book_D,"[500, 400, 600]","[NaN, Japan, Canada]"
4,Book_E,[987],[Japan]


This is what the user wanted:
    
> My output dataframe should be a one to one mapping between the name of the book and one of the Book Ids with the following rule :
> If the list of Book_Ids has only one ID , pick that. If there are more than one , pick according to the following ranking : USA > Canada > Japan

## Solution

Best to break it down into steps:

In [2]:
# Function to rank items according to country

def country_rank(item, ranking=['USA', 'Canada', 'Japan']):
    try:
        i = ranking.index(item[0])
    except ValueError:
        i = len(ranking)
    return i

print(country_rank(('Canada', 999)))

1


In [3]:
# Function to combine countries and IDs and then pick 
# the one with the highest rank:

def pick_book_id(x, ranking=['USA', 'Canada', 'Japan']):
    book_ids = list(zip(x['Countries_Published'], x['Book_Ids']))
    return sorted(book_ids, key=country_rank)[0]

pick_book_id(df.loc[1])

('USA', 1122)

Put it all together:

In [4]:
df['Selected IDs'] = df.apply(pick_book_id, axis=1)
print(df)

  Book_Name         Book_Ids   Countries_Published   Selected IDs
0    Book_A            [123]                 [USA]     (USA, 123)
1    Book_B     [4670, 1122]         [Canada, USA]    (USA, 1122)
2    Book_C  [11, 200, 7688]  [Japan, USA, Canada]     (USA, 200)
3    Book_D  [500, 400, 600]  [NaN, Japan, Canada]  (Canada, 600)
4    Book_E            [987]               [Japan]   (Japan, 987)
